In [1]:
# In Class Lab 7 Template

# G. Besla
# with code from R. Hoffman and E. Patel

# Add path to other HW folders, and Data; Modular Design
import sys
sys.path.append("../Homeworks")

# import modules
import warnings
from tqdm import trange
import numpy as np
from numpy.lib.recfunctions import structured_to_unstructured as rec2arr
import astropy.units as u
from astropy.constants import G
# 1) Make plots 
from astropy.visualization import PercentileInterval, ZScaleInterval, AsymmetricPercentileInterval, PowerStretch, ContrastBiasStretch
from astropy.visualization import MinMaxInterval, SqrtStretch, ImageNormalize, LogStretch, SinhStretch, PowerDistStretch

# import plotting modules
import matplotlib
import mpl_scatter_density
from mpl_scatter_density import ScatterDensityArtist
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FFMpegWriter
import matplotlib.animation as animation
from IPython.display import HTML

# %matplotlib qt
# %matplotlib widget

# from Homework2.ReadFile import Read
from Homework4.CenterOfMass import CenterOfMass
from Homework5.MassProfile import MassProfile
from helper_funcs import RotateFrame, GetRotMatrix, SurfaceDensityProfile, getMag, updateScatters
from helper_funcs import get_snapshot, get_orbit, Cyl_Coords, Lerp, Read, JacobiRadius, smoothstep

ticks_size = 10
label_size = 12
title_size = 15

matplotlib.rcParams['animation.html'] = 'html5'
matplotlib.rcParams['xtick.labelsize'] = matplotlib.rcParams['ytick.labelsize'] = ticks_size 
matplotlib.rcParams['legend.fontsize'] = ticks_size
matplotlib.rcParams['axes.labelsize'] = label_size
matplotlib.rcParams['figure.titlesize'] = label_size

## Animation Code

This notebook is effectively an animation pipeline. It will produce the multipanel animation of the Local Group with a focus on looking at the stellar streams of M33.
In order to fully understand the final product of the pipeline you need to read the paper that comes with this notebook. 
After multiple iterations I realized it was best to separate a script module with all the functions needed to make this notebook easier to read. The script is called _helper_funcs.py_.
This notebook will only include documentation on the steps that are taken to produce the described visualization pipeline.

### Initilize the first snapshot arrays

The visualization pipeline must first be initialized with the first snapshot. This is because we want to initialize the figures, and matplotlib artistis.
We begin by taking in all the snapshot data $\mathrm{r} = \left[\mathrm{x}, \mathrm{y}, \mathrm{z}\right]$ and $\mathrm{v} = \left[\mathrm{v_x}, \mathrm{v_y}, \mathrm{v_z}\right]$ for every galaxy. Then we load in the orbit data (which is already available in the repo as well) for every galaxy with all the 802 snapshots center of mass information.

_Notes:_
- We are using the low resolution data.
- We are only working with luminous (baryon) data, so no need of Dark Matter (actually a lie, you'll see later).
- The current state of the notebook will not display any graphical interface, because of the `matplotlib.use("Agg")` line that optimizes my animation (by not showing anything to me).

In [2]:
snap=0

# Load in galaxy data, separate baryons, and position and velocity vectors
mw_data, mw_baryons, mw_r, mw_v = get_snapshot('mw', baryons=True, snap=snap)
m31_data, m31_baryons, m31_r, m31_v = get_snapshot('m31', baryons=True, snap=snap)
m33_data, m33_baryons, m33_r, m33_v = get_snapshot('m33', baryons=True, snap=snap)

# Center of Mass info from Orbits
mw_com, mw_com_r, mw_com_v = get_orbit('mw', snap=snap)
m31_com, m31_com_r, m31_com_v = get_orbit('m31', snap=snap)
m33_com, m33_com_r, m33_com_v = get_orbit('m33', snap=snap)

time = mw_com['t'][snap] * u.Myr

### Make transformations
Here we simply transform the original-frame galaxy particle vectors to center-of-mass frame for every respective galaxy.
I modified the RotateFrame function such that it will take multiple $\mathrm r$ and $\mathrm v$ pair vectors, and it will rotate every single one appropriately (top view).
In the last line we are just getting the $r$-magnitudes for each galaxy particle. However, in the three lines before we are doing the following:

- `mw_m33_r` is the Milky Way particle positions w.r.t. M33's CoM. 
- `m31_m33_r` is M31 particle positions w.r.t. M33's CoM. 
- `m33_m31_r` is M33 particle positions w.r.t. M31's CoM. 

The last one is used to display the stellar streams orbiting M31.

In [3]:
# Transform particle vectors to CoM per Galaxy; 
# Shapes: (3, Nparticles)
mw_rcom = mw_r - mw_com_r
mw_vcom = mw_v - mw_com_v
m31_rcom = m31_r - m31_com_r
m31_vcom = m31_v - m31_com_v
m33_rcom = m33_r - m33_com_r
m33_vcom = m33_v - m33_com_v

mw_top_rcom, m31_top_rcom, m33_top_rcom = RotateFrame(mw_rcom, mw_vcom, m31_rcom, m31_vcom, m33_rcom, m33_vcom)

mw_m33_r = mw_r - m33_com_r
m31_m33_r = m31_r - m33_com_r
m33_m31_r = m33_r - m31_com_r

# Get R magnitudes of each galaxy particles with respect to its CoM
mw_rmag, m31_rmag, m33_rmag, m33_m31_rmag = getMag(mw_rcom, m31_rcom, m33_rcom, m33_m31_r)

### Convert, Index, and Jacobi

Here we are doing a few other things. 
- First we define radii values per galaxy that enclose 99.5% of their respective baryon particles. These percentiles are used to plot the data; we want to plot most of the particles, but not all outliers.
- Then we calculate each galaxy's cylindrical coordinates from their top-view rotated frames. 
- Third we calculate the Jacobi Radius using. In this implementation we use M33 and M31's both DM and baryon enclosed masses... and we are using D/2 as the point mass radius of M33.
- Fourth, we simply do $r_{str} = D - r_J$ as discussed in the paper; the radius at which unbounded M33 particles will be w.r.t M31.
- Finally we create indice arrays for each case; the galaxy plotting selections, the tidal particles, the stream particles, and the non-tidal partcles of M33.

In [4]:
# Finally create an indexing array removing galaxy "outlier" particles
mw_rmax = np.percentile(mw_rmag, 99.5)
m31_rmax = np.percentile(m31_rmag, 99.5)
m33_rmax = np.percentile(m33_rmag, 99.5)

# Calculate cylindrical coordinates of each top-view galaxy frame
mw_cyl, m31_cyl, m33_cyl = Cyl_Coords(mw_top_rcom, m31_top_rcom, m33_top_rcom)

# Calculate the jacobi radius of the M33-M31 System
m31_jac_radius = JacobiRadius(m33_com_r, m31_com_r, m33_data, m31_data)

# Calculate D - rj
D = np.sqrt(((m33_com_r - m31_com_r)**2).sum())
m31_stream_rad = D - m31_jac_radius

# Array indices for streams
isTidal = m33_rmag > m31_jac_radius
isStream = isTidal & (m33_m31_rmag < m31_stream_rad)
notTidal = m33_rmag < m31_jac_radius

m33_tidal = np.where(isTidal)
m33_streams, = np.where(isStream)
m33_nottidal, = np.where(notTidal)

# Selection arrays
mw_pselection, = np.where(mw_rmag < mw_rmax)
m31_pselection, = np.where(m31_rmag < m31_rmax)
m33_pselection, = np.where(m33_rmag < m33_rmax)

### Test canvas/plotting configuration

The following code cell's only purpuse was to be re-run while I was experimenting with the canvas, and perfecting the plot positions, as well as aesthetics.

In [7]:
init_xlims = -210, 490
init_ylims = -430, 270

final_xlims = -250, 250
final_ylims = -250, 250

init_rho = 45, 15, 55
final_rho = 120, 280, 140

fig = plt.figure(constrained_layout=False, figsize=(7,7))
subplots_kwargs = dict(projection='scatter_density', xticks=[], fc='k', yticks=[], aspect=1, adjustable='box')

gs1 = fig.add_gridspec(nrows=1, ncols=3, left=0.05, right=0.98, top=0.99, bottom=0.60, wspace=0.24, hspace=0.05)

ax1 = fig.add_subplot(gs1[0, 0], **subplots_kwargs, title='Milky Way')
ax2 = fig.add_subplot(gs1[0, 1], **subplots_kwargs, title='Triangulum')
ax3 = fig.add_subplot(gs1[0, 2], **subplots_kwargs, title='Andromeda')

gs2 = fig.add_gridspec(nrows=1, ncols=3, left=0.05, right=0.98, top=0.64, bottom=0.54, wspace=0.18, hspace=0.05)

ax4 = fig.add_subplot(gs2[0, 0], xticks=[], yticks=[])
ax5 = fig.add_subplot(gs2[0, 1], xticks=[], yticks=[])
ax6 = fig.add_subplot(gs2[0, 2], xticks=[], yticks=[])

gs3 = fig.add_gridspec(nrows=1, ncols=2, left=0.055, right=0.98, top=0.54, bottom=0.02, wspace=0.05, hspace=0.05)
ax7 = fig.add_subplot(gs3[0, 0], **subplots_kwargs)
ax8 = fig.add_subplot(gs3[0, 1], **subplots_kwargs)

axes = ax1, ax2, ax3, ax7
rho_axes = ax4, ax5, ax6

for ax in axes[:3]+rho_axes:
    ax.tick_params(axis='both', which='major', pad=0)

mw_rho_norm = ImageNormalize(mw_cyl[0][mw_pselection], interval=MinMaxInterval(), stretch=LogStretch())
mw_scatt = ax4.scatter(mw_cyl[1][mw_pselection], mw_cyl[0][mw_pselection], ec="None", fc='m', s=np.log10(mw_cyl[0][mw_pselection] + 1.0),
                       c=mw_cyl[0][mw_pselection], alpha=0.7, norm=mw_rho_norm, cmap='magma_r')
m33_rho_norm = ImageNormalize(m33_cyl[0][m33_pselection], interval=MinMaxInterval(), stretch=LogStretch())
m33_scatt = ax5.scatter(m33_cyl[1][m33_pselection], m33_cyl[0][m33_pselection], ec="None", fc='m', s=np.log10(m33_cyl[0][m33_pselection] + 1.0),
                        c=m33_cyl[0][m33_pselection], alpha=0.7, norm=m33_rho_norm, cmap='viridis_r')
m31_rho_norm = ImageNormalize(m31_cyl[0][m31_pselection], interval=MinMaxInterval(), stretch=LogStretch())
m31_scatt = ax6.scatter(m31_cyl[1][m31_pselection], m31_cyl[0][m31_pselection], ec="None", fc='m', s=np.log10(m31_cyl[0][m31_pselection] + 1.0),
                        c=m31_cyl[0][m31_pselection], alpha=0.7, norm=m31_rho_norm, cmap='hot_r')

mw_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(4e3))
mw_density = ax1.scatter_density(mw_top_rcom[0][mw_pselection], mw_top_rcom[1][mw_pselection], norm=mw_norm, cmap='magma')

m33_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(6e3))
m33_density = ax2.scatter_density(m33_top_rcom[0][m33_nottidal], m33_top_rcom[1][m33_nottidal], norm=m33_norm, cmap='cviridis')

m33_stream_norm = ImageNormalize(interval=AsymmetricPercentileInterval(95, 100), stretch=LogStretch(400))
m33_stream_den = ScatterDensityArtist(ax2, m33_top_rcom[0][m33_tidal], m33_top_rcom[1][m33_tidal], 
                                      dpi=72, norm=m33_stream_norm, color='#f28af2', alpha=0.65)
ax2.add_artist(m33_stream_den)
    
m31_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(4e3))
m31_density = ax3.scatter_density(m31_top_rcom[0][m31_pselection], m31_top_rcom[1][m31_pselection], norm=m31_norm, cmap='hot')

density3 = ax7.scatter_density(m33_rcom[1][m33_pselection], m33_rcom[0][m33_pselection], cmap='cviridis', norm=m33_norm)
density1 = ax7.scatter_density(mw_m33_r[1][mw_pselection], mw_m33_r[0][mw_pselection], cmap='cmagma', norm=mw_norm)
density2 = ax7.scatter_density(m31_m33_r[1][m31_pselection], m31_m33_r[0][m31_pselection], cmap='chot', norm=m31_norm)

m31_core_density = ax8.scatter_density(m31_rcom[0][m31_pselection], m31_rcom[1][m31_pselection], norm=m31_norm, cmap='chot', alpha=0.2)
m33_m31_stream_den = ScatterDensityArtist(ax8, m33_m31_r[1][m33_streams], m33_m31_r[0][m33_streams], 
                                      dpi=72, cmap='ccividis', norm=m33_stream_norm)
ax8.add_artist(m33_m31_stream_den)

line1 = ax5.axhline(y=m31_jac_radius, lw=2, ls='-', c='#2eb832', alpha=0.7)

rho_size = init_rho

for i in range(3):
    win_size = rho_size[i]
    axes[i].set_xlim(-win_size, win_size)
    axes[i].set_ylim(-win_size, win_size)
    axes[i].set_yticks([-win_size, win_size])
    
    rho_axes[i].set_ylim(win_size, 0)
    if i==1 and m31_jac_radius < win_size:
        rho_axes[i].set_yticks([round(m31_jac_radius),])
    else:
        rho_axes[i].set_yticks([round(0.75*win_size)])

winxlim = init_xlims
winylim = init_ylims

ax7.set_ylim(*winxlim)
ax7.set_xlim(*winylim)
ax7.set_yticks([round(Lerp(winxlim[0], winxlim[1],0.25)), round(Lerp(winxlim[0], winxlim[1],0.75))])
ax7.set_xticks([round(Lerp(winylim[0], winylim[1],0.25)), round(Lerp(winylim[0], winylim[1],0.75))])

str_ylims = 60
ax8.set_ylim(-str_ylims, str_ylims)
ax8.set_xlim(-str_ylims, str_ylims)
ax8.set_xticks([-50, 50])

ann  = ax7.annotate(f'Time: {time}', (0.01, 0.92), xycoords='axes fraction', fontsize=10, animated=False, color='white')

# plt.tight_layout()

plt.show()

Same goes for the coming two cells!!!

In [22]:
m31_core_density.set_xy(m31_top_rcom[0][m31_rmag < m31_rsmall], m31_top_rcom[1][m31_rmag < m31_rsmall])
str_ylims = 100
ax8.set_ylim(-str_ylims, str_ylims)
ax8.set_xlim(-str_ylims, str_ylims)
ax8.set_xticks([-50, 50])

In [305]:
width = (m33_stream_den._ax.get_position().width *
         m33_stream_den._ax.figure.get_figwidth())
height = (m33_stream_den._ax.get_position().height *
          m33_stream_den._ax.figure.get_figheight())

nx = int(round(width * 72))
ny = int(round(height * 72))
bins = nx,ny

xmin, xmax = m33_stream_den._ax.get_xlim()
ymin, ymax = m33_stream_den._ax.get_ylim()

m33_stream_norm = ImageNormalize(interval=AsymmetricPercentileInterval(98, 100), stretch=LogStretch(400))
m33_stream_norm._set_limits(m33_stream_den._array_func(bins=bins, range=((ymin, ymax), (xmin, xmax))))
m33_stream_norm.vmin, m33_stream_norm.vmax, m33_stream_norm(8).data[0]

(2.0, 14.0, 0.8847746139804832)

### Animation Visualization Pipeline

Now this cell will take care of everything. The only pre-requisite is to have run all the code cells above, with exception to the testing code cells. I mean, they're actually optional it's not like they're gonan ruin everything. Just keep in mind that the current notebook will not display any plots, as noted at the beginning of the notebook. 
I will leave comments below going over everything so let's get to it.

In [5]:
# Initialize the movie/animation stuff. The important attributes is the 
# FPS (frames per second) of the movie and the FFMpegWriter which will 
# stick all images into a video

FPS = 15
metadata = dict(title='Movie Test', artist='Matplotlib',
                comment='Movie support!')
writer = FFMpegWriter(fps=FPS, metadata=metadata)

# Okay so now let's initialize the canvas we'll be drawing onto. 
# We basically have eight axes. In order to best organize our canvas we actually
# generate three different GridSpecs. It is easier to manipulate the GridSpecs 
# positions, than each axes individually.
fig = plt.figure(constrained_layout=False, figsize=(7,7))

# These keyword arguments are used to initialize the scatter density plots that
# display more than one scatter_density artist.
subplots_kwargs = dict(projection='scatter_density', xticks=[], fc='k', yticks=[], aspect=1, adjustable='box')

# Each GridSpec has customized left, right, top, bottom wspace, hspace arguments
# completely determined through itertation and experimentation.
gs1 = fig.add_gridspec(nrows=1, ncols=3, left=0.05, right=0.98, top=0.99, bottom=0.60, wspace=0.24, hspace=0.05)

# The first GridSpec will hold the three top axes of the top view galaxies
ax1 = fig.add_subplot(gs1[0, 0], **subplots_kwargs, title='Milky Way')
ax2 = fig.add_subplot(gs1[0, 1], **subplots_kwargs, title='Triangulum')
ax3 = fig.add_subplot(gs1[0, 2], **subplots_kwargs, title='Andromeda')

gs2 = fig.add_gridspec(nrows=1, ncols=3, left=0.05, right=0.98, top=0.64, bottom=0.54, wspace=0.18, hspace=0.05)

# The second GridSpec will hold the three middle cylindrical view plots, these
# will be simple scatter plots so no need to pass the subplots_kwargs arguments
ax4 = fig.add_subplot(gs2[0, 0], xticks=[], yticks=[])
ax5 = fig.add_subplot(gs2[0, 1], xticks=[], yticks=[])
ax6 = fig.add_subplot(gs2[0, 2], xticks=[], yticks=[])

# And the last GridSpec will hold the bottom two plots. 
gs3 = fig.add_gridspec(nrows=1, ncols=2, left=0.055, right=0.98, top=0.54, bottom=0.02, wspace=0.05, hspace=0.05)
ax7 = fig.add_subplot(gs3[0, 0], **subplots_kwargs)
ax8 = fig.add_subplot(gs3[0, 1], **subplots_kwargs)

# These two are simple iterators for our axes
axes = ax1, ax2, ax3, ax7
rho_axes = ax4, ax5, ax6

# The following tuples will be used as inital, and final
# ylimiters and xlimiters. This animation also animates the x-y axis.
init_xlims = -210, 490
init_ylims = -430, 270

final_xlims = -250, 250
final_ylims = -250, 250

init_rho = 45, 15, 55
final_rho = 120, 280, 140

# Remove some space off my backyard
for ax in axes[:3]+rho_axes:
    ax.tick_params(axis='both', which='major', pad=0)

# The next six lines are as follows:
# Each rho_norm and scatt pair go together. The rho_norm variables are 
# Astropy Image Normalizers  to better visualize the scatter plots for these people.
# The scatter plots are using the respective normalizers, as well as size scalers
# We actually explore the size scalers
mw_rho_norm = ImageNormalize(mw_cyl[0][mw_pselection], interval=MinMaxInterval(), stretch=LogStretch())
mw_scatt = ax4.scatter(mw_cyl[1][mw_pselection], mw_cyl[0][mw_pselection], ec="None", fc='m', s=np.log10(mw_cyl[0][mw_pselection] + 1.0),
                       c=mw_cyl[0][mw_pselection], alpha=0.7, norm=mw_rho_norm, cmap='magma_r')
m33_rho_norm = ImageNormalize(m33_cyl[0][m33_pselection], interval=MinMaxInterval(), stretch=LogStretch())
m33_scatt = ax5.scatter(m33_cyl[1][m33_pselection], m33_cyl[0][m33_pselection], ec="None", fc='m', s=np.log10(m33_cyl[0][m33_pselection] + 1.0),
                        c=m33_cyl[0][m33_pselection], alpha=0.7, norm=m33_rho_norm, cmap='viridis_r')
m31_rho_norm = ImageNormalize(m31_cyl[0][m31_pselection], interval=MinMaxInterval(), stretch=LogStretch())
m31_scatt = ax6.scatter(m31_cyl[1][m31_pselection], m31_cyl[0][m31_pselection], ec="None", fc='m', s=np.log10(m31_cyl[0][m31_pselection] + 1.0),
                        c=m31_cyl[0][m31_pselection], alpha=0.7, norm=m31_rho_norm, cmap='hot_r')

mw_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(4e3))
mw_density = ax1.scatter_density(mw_top_rcom[0][mw_pselection], mw_top_rcom[1][mw_pselection], norm=mw_norm, cmap='magma')

m33_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(6e3))
m33_density = ax2.scatter_density(m33_top_rcom[0][m33_nottidal], m33_top_rcom[1][m33_nottidal], norm=m33_norm, cmap='cviridis')

m33_stream_norm = ImageNormalize(interval=AsymmetricPercentileInterval(95, 100), stretch=LogStretch(400))
m33_stream_den = ScatterDensityArtist(ax2, m33_top_rcom[0][m33_tidal], m33_top_rcom[1][m33_tidal], 
                                      dpi=72, norm=m33_stream_norm, color='#f28af2', alpha=0.65)
ax2.add_artist(m33_stream_den)
    
m31_norm = ImageNormalize(interval=MinMaxInterval(), stretch=LogStretch(4e3))
m31_density = ax3.scatter_density(m31_top_rcom[0][m31_pselection], m31_top_rcom[1][m31_pselection], norm=m31_norm, cmap='hot')

density3 = ax7.scatter_density(m33_rcom[1][m33_pselection], m33_rcom[0][m33_pselection], cmap='cviridis', norm=m33_norm)
density1 = ax7.scatter_density(mw_m33_r[1][mw_pselection], mw_m33_r[0][mw_pselection], cmap='cmagma', norm=mw_norm)
density2 = ax7.scatter_density(m31_m33_r[1][m31_pselection], m31_m33_r[0][m31_pselection], cmap='chot', norm=m31_norm)

m31_core_density = ax8.scatter_density(m31_rcom[0][m31_pselection], m31_rcom[1][m31_pselection], norm=m31_norm, cmap='chot', alpha=0.2)
m33_m31_stream_den = ScatterDensityArtist(ax8, m33_m31_r[1][m33_streams], m33_m31_r[0][m33_streams], 
                                      dpi=72, cmap='ccividis', norm=m33_stream_norm)
ax8.add_artist(m33_m31_stream_den)

line1 = ax5.axhline(y=m31_jac_radius, lw=2, ls='-', c='#2eb832', alpha=0.7)

for i in range(3):
    win_size = init_rho[i]
    axes[i].set_xlim(-win_size, win_size)
    axes[i].set_ylim(-win_size, win_size)
    axes[i].set_yticks([-win_size, win_size])
    
    rho_axes[i].set_ylim(win_size, 0)
    if i==1 and m31_jac_radius < 0.75*win_size:
        rho_axes[i].set_yticks([round(m31_jac_radius),])
    else:
        rho_axes[i].set_yticks([round(0.75*win_size)])

ax7.set_ylim(*init_xlims)
ax7.set_xlim(*init_ylims)
ax7.set_yticks([round(Lerp(init_xlims[0], init_xlims[1],0.25)), round(Lerp(init_xlims[0], init_xlims[1],0.75))])
ax7.set_xticks([round(Lerp(init_ylims[0], init_ylims[1],0.25)), round(Lerp(init_ylims[0], init_ylims[1],0.75))])

str_ylims = 60
ax8.set_ylim(-str_ylims, str_ylims)
ax8.set_xlim(-str_ylims, str_ylims)
ax8.set_xticks([-50, 50])

ann  = ax7.annotate(f'Time: {time}', (0.01, 0.92), xycoords='axes fraction', fontsize=10, animated=False, color='white')

secs_move = 1.0
rho_xylims = [Lerp(init, final) for init, final in zip(init_rho, final_rho)]
win_xlims = [Lerp(init, final) for init, final in zip(init_xlims, final_xlims)]
win_ylims = [Lerp(init, final) for init, final in zip(init_ylims, final_ylims)]

# frames_move = round(FPS * secs_move)
# curr_frame = frames_move + 1
with writer.saving(fig, "writer_test19.mp4", 100):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        for n in trange(1, 802, 1):
            writer.grab_frame()
            # Load in galaxy data, separate baryons, and position and velocity vectors
            mw_data, mw_baryons, mw_r, mw_v = get_snapshot('mw', baryons=True, snap=n)
            m31_data, m31_baryons, m31_r, m31_v = get_snapshot('m31', baryons=True, snap=n)
            m33_data, m33_baryons, m33_r, m33_v = get_snapshot('m33', baryons=True, snap=n)

            # Center of Mass info from Orbits

            mw_com_r, mw_com_v = get_orbit(mw_com, snap=n)
            m31_com_r, m31_com_v = get_orbit(m31_com, snap=n)
            m33_com_r, m33_com_v = get_orbit(m33_com, snap=n)

            time = mw_com['t'][n] * u.Myr

            # Transform particle vectors to CoM per Galaxy; 
            # Shapes: (3, Nparticles)
            mw_rcom = mw_r - mw_com_r
            mw_vcom = mw_v - mw_com_v
            m31_rcom = m31_r - m31_com_r
            m31_vcom = m31_v - m31_com_v
            m33_rcom = m33_r - m33_com_r
            m33_vcom = m33_v - m33_com_v

            mw_top_rcom, m31_top_rcom, m33_top_rcom = RotateFrame(mw_rcom, mw_vcom, m31_rcom, 
                                                                  m31_vcom, m33_rcom, m33_vcom)

            mw_m33_r = mw_r - m33_com_r
            m31_m33_r = m31_r - m33_com_r
            m33_m31_r = m33_r - m31_com_r

            # Get R magnitudes of each galaxy particles with respect to its CoM
            mw_rmag, m31_rmag, m33_rmag, m33_m31_rmag = getMag(mw_rcom, m31_rcom, m33_rcom, m33_m31_r)

            # Finally create an indexing array removing galaxy "outlier" particles
            mw_rmax = np.percentile(mw_rmag, 99.5)
            m31_rmax = np.percentile(m31_rmag, 99.5)
            m33_rmax = np.percentile(m33_rmag, 99.5)

            # Calculate cylindrical coordinates of each top-view galaxy frame
            mw_cyl, m31_cyl, m33_cyl = Cyl_Coords(mw_top_rcom, m31_top_rcom, m33_top_rcom)

            # Calculate the jacobi radius of the M33-M31 System
            m31_jac_radius = JacobiRadius(m33_com_r, m31_com_r, m33_data, m31_data)

            # Calculate D - rj
            D = np.sqrt(((m33_com_r - m31_com_r)**2).sum())
            m31_stream_rad = D - m31_jac_radius

            # Array indices for streams
            isTidal = m33_rmag > m31_jac_radius
            isStream = isTidal & (m33_m31_rmag < m31_stream_rad)
            notTidal = m33_rmag < m31_jac_radius

            m33_tidal, = np.where(isTidal)
            m33_streams, = np.where(isStream)
            # m33_new_streams, = np.where(isStream)
            m33_nottidal, = np.where(notTidal)
            
            # Add any new stream index to the permanent list of m33_streams
            # m33_streams = np.union1d(m33_streams, m33_new_streams)

            # Selection arrays
            mw_pselection, = np.where(mw_rmag < mw_rmax)
            m31_pselection, = np.where(m31_rmag < m31_rmax)
            m33_pselection, = np.where(m33_rmag < m33_rmax)
            
            if n%10==0:
                m31_norm.vmax, m31_norm.vmin = None, None
                m33_norm.vmax, m33_norm.vmin = None, None
                mw_norm.vmax, mw_norm.vmin = None, None
                m33_stream_norm.vmax, m33_stream_norm.vmin = None, None
            
            updateScatters(mw_scatt, mw_cyl, mw_pselection, m31_scatt, m31_cyl, 
                           m31_pselection, m33_scatt, m33_cyl, m33_pselection)

            if m33_streams.size > 0:
                m33_stream_den.set_xy(m33_top_rcom[0][m33_tidal], m33_top_rcom[1][m33_tidal])
                m33_m31_stream_den.set_xy(m33_m31_r[1][m33_streams], m33_m31_r[0][m33_streams])
            
            m31_core_density.set_xy(m31_rcom[0][m31_pselection], m31_rcom[1][m31_pselection])

            mw_density.set_xy(mw_top_rcom[0][mw_pselection], mw_top_rcom[1][mw_pselection])
            m31_density.set_xy(m31_top_rcom[0][m31_pselection], m31_top_rcom[1][m31_pselection])
            m33_density.set_xy(m33_top_rcom[0][m33_nottidal], m33_top_rcom[1][m33_nottidal])

            density3.set_xy(m33_rcom[0][m33_pselection], m33_rcom[1][m33_pselection])
            density2.set_xy(m31_m33_r[1][m31_pselection], m31_m33_r[0][m31_pselection])
            density1.set_xy(mw_m33_r[1][mw_pselection], mw_m33_r[0][mw_pselection])

            ann.set_text(f'Time: {time}')

            win_ylim = win_ylims[0][n], win_ylims[1][n]
            win_xlim = win_xlims[0][n], win_xlims[1][n]

            ax7.set_ylim(*win_xlim)
            ax7.set_xlim(*win_ylim)
            ax7.set_yticks([round(Lerp(win_xlim[0], win_xlim[1],0.25)), round(Lerp(win_xlim[0], win_xlim[1],0.75))])
            ax7.set_xticks([round(Lerp(win_ylim[0], win_ylim[1],0.25)), round(Lerp(win_ylim[0], win_ylim[1],0.75))])

            for i in range(3):
                new_lim = rho_xylims[i][n]
                axes[i].set_xlim(-new_lim, new_lim)
                axes[i].set_ylim(-new_lim, new_lim)
                axes[i].set_yticks([-np.floor(new_lim), np.floor(new_lim)])

                rho_axes[i].set_ylim(bottom=new_lim)
                if (i==1) and (m31_jac_radius < 0.75*new_lim):
                    rho_axes[i].set_yticks([round(m31_jac_radius)])
                else:
                    rho_axes[i].set_yticks([round(0.75*new_lim)])

c:\users\rayzote\workplaces\pypackages\mpl-scatter-density\mpl_scatter_density\generic_density_artist.py:79: RuntimeWarning: All-NaN slice encountered
  vmin = self._density_vmin(array)
c:\users\rayzote\workplaces\pypackages\mpl-scatter-density\mpl_scatter_density\generic_density_artist.py:84: RuntimeWarning: All-NaN slice encountered
  vmax = self._density_vmax(array)
100%|████████████████████████████████████████████████████████████████████████████████| 801/801 [30:00<00:00,  2.25s/it]
C:\Users\rayzote\AppData\Roaming\Python\Python37\site-packages\astropy\visualization\stretch.py:285: RuntimeWarning: invalid value encountered in log
  np.log(values, out=values)
C:\Users\rayzote\Miniconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
